In [62]:
import pandas as pd
import sweetviz as sv



dft = pd.read_csv("SBA_Cleaned.csv")

dft["MIS_Status"] = dft["MIS_Status"].replace({"CHGOFF": 0,"P I F": 1})



df = dft.drop(columns=["City","Bank","BankState","SBA_Appv"])


dft.head()

,City,State,Bank,BankState,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,MIS_Status,GrAppv,SBA_Appv
0,EVANSVILLE,IN,FIFTH THIRD BANK,OH,451120,84,4,2,84,84,N,0,N,Y,1,60000,48000
1,NEW PARIS,IN,1ST SOURCE BANK,IN,722410,60,2,2,60,60,N,0,N,Y,1,40000,32000
2,BLOOMINGTON,IN,GRANT COUNTY STATE BANK,IN,621210,180,7,1,180,180,N,0,N,N,1,287000,215250
3,BROKEN ARROW,OK,1ST NATL BK & TR CO OF BROKEN,OK,0,60,2,1,60,60,N,0,N,Y,1,35000,28000
4,ORLANDO,FL,FLORIDA BUS. DEVEL CORP,FL,0,240,14,1,240,240,N,0,N,N,1,229000,229000


In [63]:
df[df["RetainedJob"].notna()]

,State,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,MIS_Status,GrAppv
0,IN,451120,84,4,2,84,84,N,0,N,Y,1,60000
1,IN,722410,60,2,2,60,60,N,0,N,Y,1,40000
2,IN,621210,180,7,1,180,180,N,0,N,N,1,287000
3,OK,0,60,2,1,60,60,N,0,N,Y,1,35000
4,FL,0,240,14,1,240,240,N,0,N,N,1,229000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
885187,OH,451120,60,6,1,60,60,N,0,N,N,1,70000
885188,OH,451130,60,6,1,60,60,N,0,Y,N,1,85000
885189,CA,332321,108,26,1,108,108,N,0,N,N,1,300000
885190,HI,0,60,6,1,60,60,N,0,N,Y,0,75000


In [64]:
df = df.astype(object)
df["Term"] = pd.to_numeric(df["Term"], errors='coerce',downcast='integer')
df["NoEmp"] = pd.to_numeric(df["NoEmp"], errors='coerce',downcast='integer')
df["GrAppv"] = pd.to_numeric(df["GrAppv"], errors='coerce',downcast='integer')
#df["SBA_Appv"] = pd.to_numeric(df["SBA_Appv"], errors='coerce',downcast='integer')
df["CreateJob"] = pd.to_numeric(df["CreateJob"], errors='coerce',downcast='integer')
df["RetainedJob"] = pd.to_numeric(df["RetainedJob"], errors='coerce',downcast='integer')
df["MIS_Status"] = pd.to_numeric(df["MIS_Status"], errors='coerce',downcast='integer')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 885192 entries, 0 to 885191
Data columns (total 13 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   State          885192 non-null  object
 1   NAICS          885192 non-null  object
 2   Term           885192 non-null  int16 
 3   NoEmp          885192 non-null  int16 
 4   NewExist       885192 non-null  object
 5   CreateJob      885192 non-null  int16 
 6   RetainedJob    885192 non-null  int16 
 7   FranchiseCode  885192 non-null  object
 8   UrbanRural     885192 non-null  object
 9   RevLineCr      885192 non-null  object
 10  LowDoc         885192 non-null  object
 11  MIS_Status     885192 non-null  int8  
 12  GrAppv         885192 non-null  int32 
dtypes: int16(4), int32(1), int8(1), object(7)
memory usage: 58.2+ MB


In [65]:
df["NAICS"].astype(str)

df["NAICS"] = df["NAICS"].apply(lambda x: str(x)[:2])

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 885192 entries, 0 to 885191
Data columns (total 13 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   State          885192 non-null  object
 1   NAICS          885192 non-null  object
 2   Term           885192 non-null  int16 
 3   NoEmp          885192 non-null  int16 
 4   NewExist       885192 non-null  object
 5   CreateJob      885192 non-null  int16 
 6   RetainedJob    885192 non-null  int16 
 7   FranchiseCode  885192 non-null  object
 8   UrbanRural     885192 non-null  object
 9   RevLineCr      885192 non-null  object
 10  LowDoc         885192 non-null  object
 11  MIS_Status     885192 non-null  int8  
 12  GrAppv         885192 non-null  int32 
dtypes: int16(4), int32(1), int8(1), object(7)
memory usage: 58.2+ MB


In [76]:
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier


X = df.drop("MIS_Status", axis=1)
y = df["MIS_Status"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42,stratify=y)


categ_col = list(X.select_dtypes(include=['object', 'category']).columns)

num_col = list(X.select_dtypes(include=['float64', 'int64']).columns)

In [82]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

categorical_transformer = OneHotEncoder()

numerical_transformer = MinMaxScaler()

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categ_col),
        ('num', numerical_transformer, num_col),
        
    ])

param_grid = {'classifier__n_estimators': [100],
              'classifier__max_depth': [10],
              'classifier__min_samples_split': [20]}

model_RF = Pipeline(steps=[('preprocessor', preprocessor),
                            ('classifier', RandomForestClassifier(class_weight='balanced'))])

# n_estimators=100,max_depth=15,min_samples_split=5000,
grid_search_RF = GridSearchCV(model_RF,param_grid,cv=5,verbose=3)

# model_LR = Pipeline(steps=[('preprocessor', preprocessor),
#                             ('classifier', LogisticRegression(max_iter=1000))])


In [83]:
grid_search_RF.fit(X_train, y_train)


Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5] END classifier__max_depth=10, classifier__min_samples_split=20, classifier__n_estimators=100;, score=0.582 total time=  54.9s
[CV 2/5] END classifier__max_depth=10, classifier__min_samples_split=20, classifier__n_estimators=100;, score=0.581 total time=  52.9s
[CV 3/5] END classifier__max_depth=10, classifier__min_samples_split=20, classifier__n_estimators=100;, score=0.583 total time=  57.3s
[CV 4/5] END classifier__max_depth=10, classifier__min_samples_split=20, classifier__n_estimators=100;, score=0.579 total time=  57.2s
[CV 5/5] END classifier__max_depth=10, classifier__min_samples_split=20, classifier__n_estimators=100;, score=0.580 total time=  56.1s


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('cat',
                                                                         OneHotEncoder(),
                                                                         ['State',
                                                                          'NAICS',
                                                                          'NewExist',
                                                                          'FranchiseCode',
                                                                          'UrbanRural',
                                                                          'RevLineCr',
                                                                          'LowDoc']),
                                                                        ('num',
                                                                         MinMaxScaler(),
                                                                         [])])),
                                       ('classifier',
                                        RandomForestClassifier(class_weight='balanced'))]),
             param_grid={'classifier__max_depth': [10],
                         'classifier__min_samples_split': [20],
                         'classifier__n_estimators': [100]},
             verbose=3)

In [84]:
grid_search_RF.score(X_test,y_test)


0.578547220967013

In [86]:
import pickle as pkl

with open("model_RF.pkl", "wb") as file:
    pkl.dump(model_RF, file)

In [87]:
# from sklearn.model_selection import cross_val_score
# scores = cross_val_score(grid_search_RF, X_train, y_train, cv=5)
# print(scores)

In [88]:
# print(grid_search_RF.best_params_)

In [89]:
from sklearn.metrics import classification_report

y_pred = grid_search_RF.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.26      0.76      0.39     15579
           1       0.91      0.54      0.68     72941

    accuracy                           0.58     88520
   macro avg       0.59      0.65      0.53     88520
weighted avg       0.80      0.58      0.63     88520



In [74]:
importances = model_RF.named_steps['classifier'].feature_importances_

importances

array([3.23006692e-04, 8.99217131e-05, 3.33062063e-04, 3.19155915e-04,
       8.66787455e-03, 9.92323326e-05, 6.89004342e-04, 4.20833552e-04,
       5.99182956e-05, 1.99717115e-02, 4.87619878e-03, 9.18562242e-05,
       6.93823184e-05, 1.15672960e-04, 2.57597135e-03, 6.60180630e-05,
       2.10607036e-04, 7.46104589e-04, 9.80247600e-05, 4.32834666e-03,
       8.54310670e-04, 5.51948794e-04, 4.48302649e-04, 3.41979686e-03,
       1.05757675e-04, 6.03707384e-04, 4.00490750e-03, 3.72635253e-04,
       1.95350515e-03, 1.65693575e-04, 2.19400974e-03, 1.61364914e-04,
       3.44476184e-04, 3.50314224e-04, 1.79116538e-04, 1.50104962e-04,
       2.16897989e-04, 1.02263364e-04, 3.45767940e-03, 1.90503593e-03,
       6.04083623e-05, 7.28926389e-04, 5.43706722e-04, 4.00616557e-03,
       6.52529415e-04, 8.09561712e-05, 9.96100341e-04, 9.98210888e-04,
       8.36888083e-04, 2.93479499e-04, 2.84815191e-04, 1.33114301e-01,
       2.75924600e-03, 1.22214431e-04, 2.38496595e-05, 4.69287575e-03,
      

In [75]:
model_LR.fit(X_train, y_train)

NameError: name 'model_LR' is not defined

In [ ]:
model_LR.score(X_train, y_train)


0.8239688315943023

In [ ]:
model_LR.score(X_test,y_test)

0.824100904320517